In [94]:
import torch.nn as nn

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # encoder
        self.input = nn.Linear(784,512)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        
        self.fc1 = nn.Linear(512,128)
        self.fc2 = nn.Linear(128,32)
        self.fc3 = nn.Linear(32,10)
        
        # decoder
        self.up_fc1 = nn.Linear(10,32)
        self.up_fc2 = nn.Linear(32,128)
        self.up_fc3 = nn.Linear(128,512)
        self.up_fc4 = nn.Linear(512,784)
        self.sigmoid = nn.Sigmoid()

        
    def encoder(self, x):
        x = self.input(x)
        x = self.relu(x)
        x = self.tanh(x) 
        x = self.fc1(x)
        x = self.relu(x)
        x = self.tanh(x) 
        x = self.fc2(x)
        x = self.tanh(x) 
        x = self.fc3(x)
        return x
        
    def decoder(self, x):
        x = self.up_fc1(x)
        x = self.tanh(x)
        x = self.up_fc2(x)
        x = self.tanh(x)
        x = self.up_fc3(x)
        x = self.tanh(x)
        x = self.up_fc4(x)
        x = self.sigmoid(x)
        return x
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [95]:
def custom_loss(outputs, inputs, model_parameters):
    mse_loss = nn.MSELoss()(outputs, inputs)
    l2_penalty = 0.00001  * sum([(p**2).sum() for p in model_parameters])
    loss = mse_loss + l2_penalty
    return loss

In [96]:
import torch.optim as optim

autoencoder = Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.003)

In [97]:
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=12, shuffle=True)

In [98]:
import torch

class KMeans:
    def __init__(self, n_clusters, max_iter=100):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.centroids = None

    def fit(self, X):
        initial_indices = torch.randperm(X.size(0))[:self.n_clusters]
        self.centroids = X[initial_indices]

        for _ in range(self.max_iter):
            labels = self._assign_clusters(X)
            new_centroids = self._update_centroids(X, labels)
            if torch.all(torch.eq(self.centroids, new_centroids)):
                break

            self.centroids = new_centroids

    def _assign_clusters(self, X):
        distances = torch.norm(X.unsqueeze(1) - self.centroids, dim=2)
        return torch.argmin(distances, dim=1)

    def _update_centroids(self, X, labels):
        new_centroids = torch.zeros((self.n_clusters, X.size(1)), device=X.device)
        for i in range(self.n_clusters):
            new_centroids[i] = X[labels == i].mean(dim=0)
        return new_centroids

In [99]:
mnist_data = []
targets = []
for data in train_loader:
    inputs, target = data
    inputs = inputs.view(inputs.size(0), -1)
    targets.append(target)
    mnist_data.append(inputs)

mnist_data = torch.cat(mnist_data, dim=0)
targets = torch.cat(targets, dim=0)


In [100]:
torch.manual_seed(42)

kmeans = KMeans(n_clusters=10)
kmeans.fit(mnist_data)

cluster_assignments = kmeans._assign_clusters(mnist_data)
final_centroids = kmeans.centroids


In [101]:
from sklearn.metrics import adjusted_rand_score

ari = adjusted_rand_score(cluster_assignments.numpy(), targets.numpy())

print("Adjusted Rand Index:", ari)

Adjusted Rand Index: 0.35986997680043337


In [102]:
from torch.optim.lr_scheduler import StepLR

num_epochs = 50
autoencoder.to("cuda")

lr_scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

for epoch in range(num_epochs):
    running_loss = 0.0

    for data in train_loader:
        inputs, _ = data
        inputs = inputs.view(-1, 784).to("cuda")
        optimizer.zero_grad()
        outputs = autoencoder(inputs)
        loss = custom_loss(outputs, inputs, autoencoder.parameters())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    lr_scheduler.step() 
    print(f"Epoch {epoch+1}, Loss:{running_loss / (len(train_loader) / 12 )}")

Epoch 1, Loss:0.6818821611374616
Epoch 2, Loss:0.5853406475454569
Epoch 3, Loss:0.5589586384445429
Epoch 4, Loss:0.522802093309164
Epoch 5, Loss:0.5091486037775874
Epoch 6, Loss:0.5051126685753464
Epoch 7, Loss:0.5013865369454026
Epoch 8, Loss:0.4952262855827808
Epoch 9, Loss:0.4898507927075028
Epoch 10, Loss:0.48754627903997894
Epoch 11, Loss:0.4206957118883729
Epoch 12, Loss:0.3979386596918106
Epoch 13, Loss:0.3889977409377694
Epoch 14, Loss:0.38427083947509527
Epoch 15, Loss:0.38122133507877587
Epoch 16, Loss:0.3792967976734042
Epoch 17, Loss:0.3778288224443793
Epoch 18, Loss:0.3767986906483769
Epoch 19, Loss:0.37581224898844956
Epoch 20, Loss:0.37495533987879753
Epoch 21, Loss:0.3670218983978033
Epoch 22, Loss:0.3662135328456759
Epoch 23, Loss:0.3659274549677968
Epoch 24, Loss:0.36562790468484163
Epoch 25, Loss:0.36531408231407403
Epoch 26, Loss:0.36499581681489945
Epoch 27, Loss:0.364659365992248
Epoch 28, Loss:0.364328302565217
Epoch 29, Loss:0.36399939545542
Epoch 30, Loss:0.363

In [109]:
mnist_data_encoded = []

for data in train_loader:
    inputs, _ = data
    
    inputs = inputs.view(-1, 784).to("cuda")
    output = autoencoder.encoder(inputs)
    output = output.view(output.size(0), -1)  
    mnist_data_encoded.append(output)

mnist_data_encoded = torch.cat(mnist_data_encoded, dim=0)


In [110]:
torch.manual_seed(42)

kmeans_encoded = KMeans(n_clusters=10)
kmeans_encoded.fit(mnist_data_encoded)

cluster_assignments_encoded = kmeans_encoded._assign_clusters(mnist_data_encoded)
final_centroids_encoded = kmeans_encoded.centroids


In [111]:
from sklearn.metrics import adjusted_rand_score

ari = adjusted_rand_score(cluster_assignments_encoded.cpu(), targets.numpy())

print("Adjusted Rand Index:", ari)

Adjusted Rand Index: 3.115718604115946e-06
